# Подготовка и загрузка данных для обучения

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = "leapGestRecog"

# Подготовка данных
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Загрузка и подготовка данных
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # По умолчанию стоит 'categorical', если у нас несколько классов
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # По умолчанию стоит 'categorical', если у нас несколько классов
    subset='validation'
)


# Обучение модели

In [11]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Создаем модель
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10, activation='softmax')  # Указываем количество классов (10 в данном случае)
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Подготовка данных
train_generator = train_generator  # Загрузите и подготовьте данные здесь

# Обучение модели
model.fit(train_generator, epochs=10, validation_data=validation_generator)


Epoch 1/10
500/500 [==============================] - 220s 433ms/step - loss: 0.3439 - accuracy: 0.9350 - val_loss: 1.3034 - val_accuracy: 0.5675
Epoch 2/10
500/500 [==============================] - 204s 409ms/step - loss: 0.0502 - accuracy: 0.9879 - val_loss: 1.3090 - val_accuracy: 0.5878
Epoch 3/10
500/500 [==============================] - 197s 394ms/step - loss: 0.0326 - accuracy: 0.9890 - val_loss: 1.3394 - val_accuracy: 0.5652
Epoch 4/10
500/500 [==============================] - 196s 392ms/step - loss: 0.0261 - accuracy: 0.9896 - val_loss: 1.4394 - val_accuracy: 0.5548
Epoch 5/10
500/500 [==============================] - 196s 392ms/step - loss: 0.0229 - accuracy: 0.9904 - val_loss: 1.4638 - val_accuracy: 0.5545
Epoch 6/10
500/500 [==============================] - 198s 396ms/step - loss: 0.0206 - accuracy: 0.9910 - val_loss: 1.4427 - val_accuracy: 0.5663
Epoch 7/10
500/500 [==============================] - 202s 403ms/step - loss: 0.0201 - accuracy: 0.9904 - val_loss: 1.4762 -

# Сохраняем модель для внедение в приложения и пишем само приложение

In [12]:
model.save("hand_gesture_model.h5")  # Сохраняем модель в файл

In [ ]:
import cv2
import dlib
import numpy as np
from tensorflow.keras.models import load_model


In [ ]:
# Загрузка предварительно обученной модели
model = load_model("hand_gesture_model.h5")  # Замените на фактический путь к вашей модели

# Захват видеопотока с веб-камеры
cap = cv2.VideoCapture(0)

# Инициализация детектора лиц Dlib
detector = dlib.get_frontal_face_detector()

while True:
    ret, frame = cap.read()

    # Детекция лиц
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    # Если лицо обнаружено, обработка изображения
    if len(faces) > 0:
        face_detected = True
        for face in faces:
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            face_roi = frame[y:y+h, x:x+w]

            # Обработка изображения (например, изменение размера, нормализация)
            processed_frame = cv2.resize(face_roi, (224, 224))  # Пример изменения размера

            # Предсказание жестов рук с использованием модели
            predictions = model.predict(np.expand_dims(processed_frame, axis=0))

            # Определение жеста с максимальной вероятностью
            gesture_class = np.argmax(predictions)

            # Вывод результата на экран
            cv2.putText(frame, f"Gesture: {gesture_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Вывод кадра на экран
    cv2.imshow('Hand Gesture Recognition', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()